# Feature Engineering con PyTS (HAR)

En este notebook se construirá conjuntos de *features* basados en transformaciones de series temporales usando **PyTS** sobre el dataset HAR (ventanas de 2.56 s, 128 pasos, 9 canales).  

**Salidas:**  
- **Resumen de Características** (`../reports/tables/pyts_feature_summary.csv`): Un archivo de metadatos que documenta la configuración y dimensiones de cada conjunto de características generado, asegurando la trazabilidad. 
- **Set de Features PAA** (`../artifacts/pyts_features_PAA.npz`): Un archivo comprimido con características basadas en Piecewise Aggregate Approximation, que capturan la forma global de las series de tiempo.

- **Set de Features BoP** (`../artifacts/pyts_features_BOP.npz`): Un archivo con características simbólicas basadas en Bag-of-Patterns, diseñadas para modelar la distribución de patrones locales y repetitivos.

- **Set de Features de Imagen** (`../artifacts/pyts_features_GAF_pooled.npz`): Un archivo opcional con características extraídas de la representación de las series como imágenes (Gramian Angular Fields), capturando su textura y dinámica 2D.

## 1. Carga y Normalización de Datos**

En este primer paso, cargamos las series de tiempo del archivo `har_processed.npz`, obteniendo `X_train` con dimensiones de muestras, pasos de tiempo y canales, junto con las etiquetas `y_train`. Posteriormente, aplicamos **normalización Z-score** de forma independiente para cada uno de los 9 canales.

Este preprocesamiento es fundamental por dos razones: primero, garantiza que las señales de todos los sensores (acelerómetros y giroscopios) se encuentren en una escala común con media 0 y desviación estándar 1, evitando que los canales con magnitudes mayores dominen artificialmente el análisis. Segundo, muchas técnicas de análisis de series de tiempo, incluyendo PAA y SAX que se utilizarán posteriormente, son sensibles a la escala de los datos, por lo que la normalización asegura su funcionamiento estable y efectivo.

In [ ]:
# ===============================
# 1. Carga de datos desde el caché
# ===============================
import numpy as np
import pandas as pd

# Ruta del archivo procesado
har_processed_path = "../data/har_processed.npz"

# Cargar
data = np.load(har_processed_path, allow_pickle=True)
X_train = data["X_train"]       # (n, 128, 9)
y_train = data["y_train"]       # (n,)
channel_names = data["channel_names"]

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Canales:", channel_names)

## 2. Diseño de Estrategias de Características con `pyts`**

Para representar las series de tiempo, se diseñaron tres estrategias complementarias que capturan diferentes aspectos de las señales de movimiento:

**Estrategia 1: Resumen Global con PAA (Piecewise Aggregate Approximation)**

PAA divide la secuencia de 128 puntos en segmentos más pequeños y calcula el promedio de cada uno. Esto captura la forma general de la señal: actividades dinámicas como caminar muestran patrones ondulatorios, mientras que actividades estáticas como sentarse son casi planas. Se genera un vector concatenado de características para los 9 canales.

**Estrategia 2: Patrones Locales con Bag-of-Patterns (BoP)**

BoP discretiza la señal en símbolos mediante SAX y cuenta la frecuencia de patrones recurrentes. Esto permite distinguir micro-patrones entre actividades similares: aunque caminar y subir escaleras son periódicas, la forma exacta de cada paso difiere. Se genera un histograma de frecuencias por canal que se concatena.

**Estrategia 3: Dinámica Visual con Imágenes (GAF o RP)**

Esta técnica convierte las series temporales en imágenes bidimensionales que revelan patrones de autocorrelación y dinámica no lineal. Se aplica solo a los canales más informativos (acelerómetros del cuerpo) y se resume mediante pooling para obtener vectores compactos que capturan la textura temporal de la señal.

## 3. Hiperparámetros y justificación

**PAA (F1):**  
- `n_segments = 32` (o 16 / 64 como sensibilidad)  
  - 128 → 32 resume 4 muestras por segmento: equilibrio entre **detalle** y **ruido**.  
  - Mantiene vector por canal manejable (32) → total 9×32 = 288 *features*.

**Bag-of-Patterns (F2):**  
- `window_size = 16` (segmentos locales suficientemente cortos para captar un “medio paso”).  
- `word_size = 8` (resumen PAA interno por ventana para robustez).  
- `n_bins = 6` (cuantización SAX suficiente sin sobrediscretizar).  
- `norm=True` (usar z-normalización previa).  
  - Estos valores son comunes en literatura SAX y funcionan bien con ventanas de 128 pasos.

**Imágenes (F3, opcional):**  
- GAF o RP con `image_size = 64` → *pooling* a `8×8` (o medias por bloques 8×8).  
- **Canales:** iniciar con `body_acc_{x,y,z}` (3) para evitar explosión dimensional.  
  - 3×(8×8)=192 *features* si se aplana, o agregar promedios por filas/columnas para vectores más pequeños.

> **Criterio pedagógico:** parámetros buscan **estabilidad**, **baja dimensionalidad** y **interpretabilidad**, evitando *overfitting* y costos excesivos.

## 4. Pipeline de construcción

1. **Normalizar** `X_train` por canal (z-score).  
2. **F1-PAA:** aplicar por canal → concatenar → `X_PAA` `(n, 9*M)`.  
3. **F2-Bag-of-Patterns:** aplicar por canal → concatenar histogramas → `X_BOP`.  
4. **F3-Imagen (opcional):** GAF/RP en 3 canales `body_acc_*` → *pooling* 8×8 → concatenar → `X_IMG`.  
5. **Chequear varianza** y remover columnas constantes.  
6. **Guardar artefactos** (`.npz` + CSV de resumen con dimensiones, tiempos y parámetros).

## 5. Validaciones rápidas (sin entrenamiento)

- **Sanity checks:** dimensiones de salida, ausencia de NaN/inf, varianza > 0.  
- **Separabilidad preliminar:** PCA/UMAP con `X_PAA` y `X_BOP` (2D) para visualizar si hay agrupamientos por actividad.  
- **Correlaciones:** matriz de correlación entre *features* de F1 y F2 para descartar redundancias extremas.

> Nota: El modelado y las métricas formales (Accuracy, F1, Confusión) se realizarán en el notebook 03.

## 6. Exportación para el paper y trazabilidad

- Guardar **resumen de *features*** (`../reports/tables/pyts_feature_summary.csv`) con:  
  - `set_name`, `n_features`, `n_segments / window_size / word_size / n_bins`, canales usados, normalización y compute_time_sec (tiempo de cómputo en segundos que tomó generar cada conjunto de features).
- Guardar **figuras** de:  
  - PCA de `X_PAA` y de `X_BOP` (coloreado por actividad).  
- Guardar artefactos:  
  - `../artifacts/pyts_features_PAA.npz` (X_PAA, y, meta)  
  - `../artifacts/pyts_features_BOP.npz` (X_BOP, y, meta)  
  - *(Opcional)* `../artifacts/pyts_features_GAF_pooled.npz`


## 7. Limitaciones y plan para tsfresh

- **PyTS** resume forma global (PAA) y patrones locales discretizados (BoP), pero no agota todos los estadísticos temporales.  
- **tsfresh** complementará con **features estadísticas masivas** (autocorrelación, entropías, percentiles, coeficientes de Fourier), seguidas de **selección de *features***.  
- En el notebook 03 (modelado), compararemos desempeño usando:  
  - Solo PyTS (F1/F2)  
  - Solo tsfresh  
  - **Combinación PyTS + tsfresh** (si la dimensionalidad y el *overfitting* lo permiten)


## 8. Conclusiones (PyTS)

- Se definieron y justificaron **tres familias** de *features* en PyTS: **PAA (compacto)**, **BoP (motivos locales)** y **GAF/RP (opcional, firma visual)**.  
- Los parámetros propuestos priorizan **robustez, baja dimensionalidad e interpretabilidad**.  
- Dejaremos listas las matrices de *features* y el resumen para integrarlas en el **notebook de modelado** y en el **paper** (sección Metodología/Experimentación).
